In [1]:
import pandas as pd,numpy as np

In [2]:
import nltk
from nltk.corpus import stopwords

In [3]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kumar\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kumar\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
df=pd.read_csv('spam.csv',encoding='latin')
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [5]:
df.drop(columns=['Unnamed: 2',	'Unnamed: 3',	'Unnamed: 4'],axis=1,inplace=True)
df.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [6]:
df.rename(columns = {'v1': 'target', 'v2': 'text'}, inplace = True)

In [7]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
df['target'] = encoder.fit_transform(df['target'])

df.head()

,target,text
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [8]:
df = df.drop_duplicates(keep = 'first')

In [9]:
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
import string
ps=PorterStemmer()
wl=WordNetLemmatizer()

In [10]:
def transform_text(text):
    text=text.lower()
    text=nltk.word_tokenize(text)
    y=[]
    for i in text:
        if i.isalnum():
            y.append(i)
    text=y[:]
    y.clear()
    for i in text:
        if i not in set(stopwords.words('english')) and i not in string.punctuation:
            y.append(i)
    text=y[:]
    y.clear()
    for i in text:
        y.append(wl.lemmatize(i))
    
    return " ".join(y)

In [11]:
transform_text('Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...')

'go jurong point crazy available bugis n great world la e buffet cine got amore wat'

In [12]:
# 'go jurong point crazy available bugis n great world la e buffet cine got amore wat'

In [13]:
df['transformed_text'] = df['text'].apply(transform_text)
df.head()

,target,text,transformed_text
0,0,"Go until jurong point, crazy.. Available only ...",go jurong point crazy available bugis n great ...
1,0,Ok lar... Joking wif u oni...,ok lar joking wif u oni
2,1,Free entry in 2 a wkly comp to win FA Cup fina...,free entry 2 wkly comp win fa cup final tkts 2...
3,0,U dun say so early hor... U c already then say...,u dun say early hor u c already say
4,0,"Nah I don't think he goes to usf, he lives aro...",nah think go usf life around though


In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer(max_features=500)

In [15]:
X=tfidf.fit_transform(df['transformed_text']).toarray()
y=df['target'].values

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test , y_train, y_test = train_test_split(X,y,test_size = 0.20, random_state = 2)

In [17]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier

In [18]:
svc = SVC(kernel= "sigmoid", gamma  = 1.0)
knc = KNeighborsClassifier()
mnb = MultinomialNB()
dtc = DecisionTreeClassifier(max_depth = 5)
lrc = LogisticRegression(solver = 'liblinear', penalty = 'l1')
rfc = RandomForestClassifier(n_estimators = 50, random_state = 2 )
abc = AdaBoostClassifier(n_estimators = 50, random_state = 2)
bc = BaggingClassifier(n_estimators = 50, random_state = 2)
etc = ExtraTreesClassifier(n_estimators = 50, random_state = 2)
gbdt = GradientBoostingClassifier(n_estimators = 50, random_state = 2)    
xgb  = XGBClassifier(n_estimators = 50, random_state = 2)

In [19]:
models = {
    'SVC': svc,
    'KNN': knc,
    'NB': mnb,
    'DT': dtc,
    'LR': lrc,
    'RF': rfc,
    'Adaboost': abc,
    'Bgc': bc,
    'ETC': etc,
    'GBDT': gbdt,
    'xgb': xgb
    
}

In [20]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error,accuracy_score

In [21]:
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [22]:
model_list=[]
accuracy_scores=[]
for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train,y_train)
    y_train_pred=model.predict(X_train)
    y_test_pred=model.predict(X_test)
    model_train_mae,model_train_rmse,model_train_r2=evaluate_model(y_train,y_train_pred)
    model_test_mae,model_test_rmse,model_test_r2=evaluate_model(y_test,y_test_pred)
    model_train_acc=accuracy_score(y_train,y_train_pred)
    model_test_acc=accuracy_score(y_test,y_test_pred)
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    print('Model performance for Training set')
    print('model train accuracy: {:.4f}'.format(model_train_acc))
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))

    print('----------------------------------')
    
    print('Model performance for Test set')
    print('model test accuracy: {:.4f}'.format(model_test_acc))

    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2))
    accuracy_scores.append(model_test_acc)
    
    print('='*35)
    print('\n')

SVC
Model performance for Training set
model train accuracy: 0.9715
- Root Mean Squared Error: 0.1689
- Mean Absolute Error: 0.0285
- R2 Score: 0.7383
----------------------------------
Model performance for Test set
model test accuracy: 0.9642
- Root Mean Squared Error: 0.1892
- Mean Absolute Error: 0.0358
- R2 Score: 0.6906


KNN
Model performance for Training set
model train accuracy: 0.9453
- Root Mean Squared Error: 0.2338
- Mean Absolute Error: 0.0547
- R2 Score: 0.4987
----------------------------------
Model performance for Test set
model test accuracy: 0.9294
- Root Mean Squared Error: 0.2657
- Mean Absolute Error: 0.0706
- R2 Score: 0.3895


NB
Model performance for Training set
model train accuracy: 0.9748
- Root Mean Squared Error: 0.1586
- Mean Absolute Error: 0.0252
- R2 Score: 0.7693
----------------------------------
Model performance for Test set
model test accuracy: 0.9681
- Root Mean Squared Error: 0.1786
- Mean Absolute Error: 0.0319
- R2 Score: 0.7240


DT
Model pe

In [26]:
pd.DataFrame(list(zip(model_list, accuracy_scores)), columns=['Model Name', 'accuracy score']).sort_values(by=["accuracy score"],ascending=False)

,Model Name,accuracy score
8,ETC,0.970986
5,RF,0.970019
10,xgb,0.970019
2,NB,0.968085
0,SVC,0.964217
4,LR,0.964217
7,Bgc,0.962282
9,GBDT,0.954545
3,DT,0.937137
1,KNN,0.929400
